In [ ]:
#!git clone --depth 1 -q https://github.com/tensorflow/tensorflow

In [1]:
# The scripts must be run from within the train directory
%cd train
# Prepare the data
#!python data_prepare.py
# Split the data by person
#!python data_split_person.py

c:\Users\hadzi\OneDrive\Desktop\Master\Magic wand\train


In [ ]:
# The scripts must be run from within the train directory
%cd train
# Prepare the data
!python data_prepare.py
# Split the data by person
!python data_split_person.py

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import datetime
import os
from data_load import DataLoader
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot


logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [3]:
def reshape_function(data, label):
  reshaped_data = tf.reshape(data, [-1, 3, 1])
  return reshaped_data, label

In [4]:
def calculate_model_size(model):
  print(model.summary())
  var_sizes = [
      np.product(list(map(int, v.shape))) * v.dtype.size
      for v in model.trainable_variables
  ]
  print("Model size:", sum(var_sizes) / 1024, "KB")

In [5]:
def build_cnn(seq_length):
  """Builds a convolutional neural network in Keras."""
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(
          8, (4, 3),
          padding="same",
          activation="relu",
          input_shape=(seq_length, 3, 1)),  # output_shape=(batch, 128, 3, 8)
      tf.keras.layers.MaxPool2D((3, 3)),  # (batch, 42, 1, 8)
      tf.keras.layers.Dropout(0.1),  # (batch, 42, 1, 8)
      tf.keras.layers.Conv2D(16, (4, 1), padding="same",
                             activation="relu"),  # (batch, 42, 1, 16)
      tf.keras.layers.MaxPool2D((3, 1), padding="same"),  # (batch, 14, 1, 16)
      tf.keras.layers.Dropout(0.1),  # (batch, 14, 1, 16)
      tf.keras.layers.Flatten(),  # (batch, 224)
      tf.keras.layers.Dense(16, activation="relu"),  # (batch, 16)
      tf.keras.layers.Dropout(0.1),  # (batch, 16)
      tf.keras.layers.Dense(4, activation="softmax")  # (batch, 4)
  ])
  model_path = os.path.join("./netmodels", "CNN")
  print("Built CNN.")
  if not os.path.exists(model_path):
    os.makedirs(model_path)
  #model.load_weights("./netmodels/CNN/weights.h5")   # what the fuck 
  return model, model_path

In [6]:
def build_lstm(seq_length):
  """Builds an LSTM in Keras."""
  model = tf.keras.Sequential([
      tf.keras.layers.Bidirectional(
          tf.keras.layers.LSTM(22),
          input_shape=(seq_length, 3)),  # output_shape=(batch, 44)
      tf.keras.layers.Dense(4, activation="sigmoid")  # (batch, 4)
  ])
  model_path = os.path.join("./netmodels", "LSTM")
  print("Built LSTM.")
  if not os.path.exists(model_path):
    os.makedirs(model_path)
  return model, model_path


In [7]:
def load_data(train_data_path, valid_data_path, test_data_path, seq_length):
  data_loader = DataLoader(train_data_path,
                           valid_data_path,
                           test_data_path,
                           seq_length=seq_length)

  data_loader.format()
  return data_loader.train_len, data_loader.train_data, data_loader.valid_len, \
      data_loader.valid_data, data_loader.test_len, data_loader.test_data

In [8]:
def build_net(args, seq_length):
  if args == "CNN":
    model, model_path = build_cnn(seq_length)
  elif args == "LSTM":
    model, model_path = build_lstm(seq_length)
  else:
    print("Please input correct model name.(CNN  LSTM)")
  return model, model_path

In [9]:
def train_net(
    model,
    model_path,  # pylint: disable=unused-argument
    train_len,  # pylint: disable=unused-argument
    train_data,
    valid_len,
    valid_data,  # pylint: disable=unused-argument
    test_len,
    test_data,
    kind,
    epochs,
    batch_size,
    steps):
  """Trains the model."""
  calculate_model_size(model)
 
  model.compile(
      optimizer="adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"])
  if kind == "CNN":
    train_data = train_data.map(reshape_function)
    test_data = test_data.map(reshape_function)
    valid_data = valid_data.map(reshape_function)
  test_labels = np.zeros(test_len)
  idx = 0
  for data, label in test_data:  # pylint: disable=unused-variable
    test_labels[idx] = label.numpy()
    idx += 1
  train_data = train_data.batch(batch_size).repeat()
  valid_data = valid_data.batch(batch_size)
  test_data = test_data.batch(batch_size)

  model.fit(
      train_data,
      epochs=epochs,
      validation_data=valid_data,
      steps_per_epoch=steps,
      validation_steps=int((valid_len - 1) / batch_size + 1),
      callbacks=[tensorboard_callback])
  loss, acc = model.evaluate(test_data)
  pred = np.argmax(model.predict(test_data), axis=1)
  confusion = tf.math.confusion_matrix(
      labels=tf.constant(test_labels),
      predictions=tf.constant(pred),
      num_classes=4)
  print(confusion)
  print("Loss {}, Accuracy {}".format(loss, acc))

  return model

  
  

In [10]:
def test_net(
    model,
    model_path,  # pylint: disable=unused-argument
    train_len,  # pylint: disable=unused-argument
    train_data,
    valid_len,
    valid_data,  # pylint: disable=unused-argument
    test_len,
    test_data,
    kind,
    epochs,
    batch_size,
    steps):
  """Trains the model."""
 
  if kind == "CNN":
    train_data = train_data.map(reshape_function)
    test_data = test_data.map(reshape_function)
    valid_data = valid_data.map(reshape_function)
  test_labels = np.zeros(test_len)
  idx = 0
  for data, label in test_data:  # pylint: disable=unused-variable
    test_labels[idx] = label.numpy()
    idx += 1
  train_data = train_data.batch(batch_size).repeat()
  valid_data = valid_data.batch(batch_size)
  test_data = test_data.batch(batch_size)


  loss, acc = model.evaluate(test_data)
  pred = np.argmax(model.predict(test_data), axis=1)
  confusion = tf.math.confusion_matrix(
      labels=tf.constant(test_labels),
      predictions=tf.constant(pred),
      num_classes=4)
  print(confusion)
  print("Loss {}, Accuracy {}".format(loss, acc))

  return model


  

In [11]:
person = "true"
seq_length = 128
kind = "CNN"
epochs = 50
batch_size = 64
steps = 1000


In [12]:
print("Start to load data...")
if person == "true":
    train_len, train_data, valid_len, valid_data, test_len, test_data = \
        load_data("./person_split/train", "./person_split/valid",
                  "./person_split/test", seq_length)
else:
    train_len, train_data, valid_len, valid_data, test_len, test_data = \
        load_data("./data/train", "./data/valid", "./data/test", seq_length)



Start to load data...
train_data_length:16325
valid_data_length:136
test_data_length:192


In [13]:
print("Start to build net...")
model, model_path = build_net(kind, seq_length)

print("Start training...")
trainedmodel = train_net(model, model_path, train_len, train_data, valid_len, valid_data,
        test_len, test_data, kind, epochs, batch_size,steps)

print("Training finished!")


Start to build net...
Built CNN.
Start training...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 3, 8)         104       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 42, 1, 8)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 42, 1, 8)          0         
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 1, 16)         528       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 1, 16)        0         
 2D)                                                             
                                                                 
 drop

In [14]:
#testing a specific model with train data

something = test_net(model, model_path, train_len, train_data, valid_len, valid_data,
        test_len, test_data, kind, epochs, batch_size,steps)  

6/6 [==============================] - 0s 3ms/step
tf.Tensor(
[[ 82   0   0   0]
 [  1  77   4   2]
 [  0   0  83   5]
 [  0   0   4 126]], shape=(4, 4), dtype=int32)
Loss 0.14156849682331085, Accuracy 0.9583333134651184


In [ ]:
def quantized_model(model,
    model_path,  # pylint: disable=unused-argument
    train_len,  # pylint: disable=unused-argument
    train_data,
    valid_len,
    valid_data,  # pylint: disable=unused-argument
    test_len,
    test_data,
    kind,
    epochs,
    batch_size,
    steps):


    quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
    q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
    q_aware_model.compile(optimizer='adam',
            loss="sparse_categorical_crossentropy",
            metrics=['accuracy'])
    if kind == "CNN":
        train_data = train_data.map(reshape_function)
        test_data = test_data.map(reshape_function)
        valid_data = valid_data.map(reshape_function)
    test_labels = np.zeros(test_len)
    idx = 0
    for data, label in test_data:  # pylint: disable=unused-variable
        test_labels[idx] = label.numpy()
        idx += 1
    train_data = train_data.batch(batch_size).repeat()
    valid_data = valid_data.batch(batch_size)
    test_data = test_data.batch(batch_size)

    q_aware_model.summary()

    calculate_model_size(model)
 
    model.fit(
    train_data,
    epochs=epochs,
    validation_data=valid_data,
    steps_per_epoch=steps,
    validation_steps=int((valid_len - 1) / batch_size + 1),
    callbacks=[tensorboard_callback])
    loss, acc = model.evaluate(test_data)
    pred = np.argmax(model.predict(test_data), axis=1)
    confusion = tf.math.confusion_matrix(
      labels=tf.constant(test_labels),
      predictions=tf.constant(pred),
      num_classes=4)
    print(confusion)
    print("Loss {}, Accuracy {}".format(loss, acc))

    return model

In [ ]:
print("Start training...")
trainedmodel = quantized_model(model, model_path, train_len, train_data, valid_len, valid_data,
        test_len, test_data, kind, epochs, batch_size,steps)

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(trainedmodel)
tflite_model = converter.convert()
# Save the model to disk
open("model-testing-evaluation.tflite", "wb").write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\hadzi\AppData\Local\Temp\tmpv9bkfxou\assets


INFO:tensorflow:Assets written to: C:\Users\hadzi\AppData\Local\Temp\tmpv9bkfxou\assets


20512

In [16]:
!xxd -i model-testing-evaluation.tflite > model-testing-evaluation.tflite.cc

In [19]:
train_len, train_data, valid_len, valid_data, test_len, test_data = \
        load_data("./person_split/train", "./person_split/valid",
                  "./person_split/test", seq_length)


train_data_length:16325
valid_data_length:136
test_data_length:192
<TensorSliceDataset element_spec=(TensorSpec(shape=(128, 3), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>


In [94]:
np.set_printoptions(suppress=True)
resultsFile = open("testing data magic wand 3.txt", "w")
for element in test_data.as_numpy_iterator(): 
  if element[1]==3: 
    for i in range(len(element[0])):
        for j in range(len(element[0][i])):
            print(element[0][i][j])
            resultsFile.write(str(round(float(element[0][i][j]),3)))
            if j != 2:
                resultsFile.write(",")
        resultsFile.write("\n")
        #print((str(element[0]))) 
    #resultsFile.write(str(element[0]))
    #resultsFile.write("\n")
   # for i in np.nditer(element): 
        #resultsFile.write(str(np.nditer(element)))
      # resultsFile.write(str(i))


345.5898227476162
-797.6524576782725
783.0911825530537
334.0361063062133
-819.526466979784
775.2132329775633
361.4005536520983
-812.2431518620061
775.2294761541941
341.4043604805144
-796.9739920268515
777.2758968127857
333.15034779416874
-801.1739609182488
764.8935578819284
351.5184602736466
-801.7273470803979
781.5289551166316
359.37337003188094
-788.3026992825767
758.8821968826044
362.5559939919044
-823.8740133603759
764.0101067413506
342.2255896171212
-814.075275790061
765.4305498149416
367.9797886113859
-803.7225036244943
773.7626852573912
357.6623630702549
-800.9463690792838
772.9628632113042
369.1870669882429
-794.1636332251454
760.8373873491175
353.85413037764465
-804.6988485359035
764.5190638276765
339.23760005544955
-804.1895822756082
758.5367439503568
335.9869598669658
-797.5222570593614
755.8247411005499
346.5483756478633
-797.2824438894341
757.8534621800741
354.6140561592942
-802.6768750633724
752.1701419144466
338.3710902604964
-819.4926815072724
760.1419344477006
354.3441

In [ ]:
train_data_quantized = train_data.map(reshape_function)
#train_data_quantized = train_data.batch(batch_size)



In [ ]:
def representative_data_gen():
 # for input_value in train_data_quantized.batch(batch_size):
  for input_value in train_data_quantized.batch(1):
    
    # Model has only one input so each data point has one element.
    yield [np.array(input_value[0]).astype(np.float32)]

In [ ]:
# Convert the model to the TensorFlow Lite format with quantization
#converter = tf.lite.TFLiteConverter.from_keras_model(trainedmodel)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#tflite_model = converter.convert()


converter = tf.lite.TFLiteConverter.from_keras_model(trainedmodel)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()

# Save the model to disk
open("model-quantized-7650.tflite", "wb").write(tflite_model_quant)

basic_model_size = os.path.getsize("model-7650-N.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("model-quantized-7650.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)

In [ ]:
!xxd -i model-quantized-7650.tflite > model-quantized-7650.cc


In [ ]:
# From there the handling of the pre-trained Person-detection model begins

from tensorflow import keras
model = keras.models.load_model('Model_CNN1_NOMOD___2022Y_10M_10D_04H_05M_20S')
model.summary()
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("Model_CNN1_NOMOD___2022Y_10M_10D_04H_05M_20S-ver2", "wb").write(tflite_model)

# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [19]:
import numpy as np



x_train = np.load('x_train.npy')

X = np.expand_dims(x_train, axis=1)
print(X.shape)


(145832, 1, 50, 3)


In [20]:
def representative_data_gen():
 # for input_value in train_data_quantized.batch(batch_size):
  for input_value in X:
    
    # Model has only one input so each data point has one element.
    #yield [np.array(input_value[0]).astype(np.float32)]
    yield [input_value]

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()

# Save the model to disk
open("Model_CNN1_NOMOD___2022Y_10M_10D_04H_05M_20S-quantized.tflite", "wb").write(tflite_model_quant)



INFO:tensorflow:Assets written to: C:\Users\hadzi\AppData\Local\Temp\tmpfc5bwkcp\assets


INFO:tensorflow:Assets written to: C:\Users\hadzi\AppData\Local\Temp\tmpfc5bwkcp\assets
c:\Users\hadzi\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


62976

In [ ]:
!xxd -i Model_CNN1_NOMOD___2022Y_10M_10D_04H_05M_20S-quantized.tflite> Model_CNN1_NOMOD___2022Y_10M_10D_04H_05M_20S-quantized.tflite.cc